In [1]:
%matplotlib inline
import os
import sys
import numpy as np
import scipy as sp
import scipy.optimize
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import rc
import pdb

mpl.rc('font', family='Times New Roman')
mpl.rcParams['font.size'] = 25.0

/Users/Jake/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import readsmart as rs
import pandas as pd
from numba import jit

In [3]:
# Relative path to spectra files
path = "spectra_files/"

In [4]:
class spectrum(object):
    def __init__(self, observation=None, star=None, planet=None, description=None, meta=None, reference=None,
                 path_to_file=None, data=None):
        self.observation=observation
        self.star=star
        self.planet=planet
        self.description=description
        self.meta=meta
        self.reference=reference
        self._path_to_file=path_to_file
        self._data=None
        if (self._data is None) and (self._path_to_file is not None):
            self._data = spectral_data.from_file(self._path_to_file)
        self.tag=observation.replace(" ", "")+"_"+star.replace(" ", "")+"_"+\
                 planet.replace(" ", "")+"_"+description.replace(" ", "")+"_"+\
                 meta.replace(" ", "")
            
    def __str__(self):
        string = self.observation+", "+self.star+", "+self.planet+", "+self.description+", "+self.meta
        return string
    
    @property
    def path_to_file(self):
        return self._path_to_file

    @path_to_file.setter
    def path_to_file(self, value):
        self._path_to_file = value
        if value is not None:
            self._data = spectral_data.from_file(value)
        else:
            self._data = None
    
    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, value):
        self._data = value

class spectral_data(object):
    def __init__(self, wavelength=None, wavenumber=None, toa_flux=None, star_flux=None, geo_albedo=None, 
                 flux_transmission=None, absorbing_radius=None):
        self.wavelength=wavelength
        self.wavenumber=wavenumber
        self.toa_flux=toa_flux
        self.star_flux=star_flux
        self.geo_albedo=geo_albedo
        self.flux_transmission=flux_transmission
        self.absorbing_radius=absorbing_radius
    
    @classmethod
    def from_file(cls, path):
        
        # Read path file
        if path.endswith(".rad"):
            # Read rad 
            wavelength, wavenumber, star_flux, toa_flux, rad_streams = rs.rad(path)
            geo_albedo = toa_flux / star_flux
            tmp = np.chararray(len(wavelength), itemsize=1)
            tmp[:] = ""
            flux_transmission, absorbing_radius = tmp, tmp
        elif path.endswith(".tran"):
            # Read tran 
            wavelength, flux_transmission, absorbing_radius = rs.tran(path)
            tmp = np.chararray(len(wavelength), itemsize=1)
            tmp[:] = ""
            wavenumber, toa_flux, star_flux, geo_albedo = tmp, tmp, tmp, tmp
        elif path.endswith(".trnst"):
            # Read trnst
            wavelength, wavenumber, absorbing_radius, tdepth = rs.trnst(path)
            tmp = np.chararray(len(wavelength), itemsize=1)
            tmp[:] = ""
            toa_flux, star_flux, geo_albedo, flux_transmission = tmp, tmp, tmp, tmp
        else:
            print "%s is an invalid path." %path

        # Return new class instance
        return cls(wavelength=wavelength, wavenumber=wavenumber, toa_flux=toa_flux, star_flux=star_flux,
                   geo_albedo=geo_albedo, flux_transmission=flux_transmission, absorbing_radius=absorbing_radius)
    

In [5]:
#"Direct_GJ876_highO2_1bar" : "photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
test = spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="FP Earth", 
               meta="1 bar; CO2/O2/CO", 
               reference="Meadows et al., 2017; Gao et al., 2015",
               path_to_file=os.path.join(path, "smart_gao_1bar_update_xsec.trnst")
              )

In [7]:
print test

Transmission, Proxima Centauri, b, FP Earth, 1 bar; CO2/O2/CO


In [297]:
spectra = [
    
    #######################
    # DIRECT SPECTROSCOPY #
    #######################
    
    #"Direct_GJ876_highO2_1bar" : "photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="GJ 876", 
               planet="FP Earth", 
               description="High O2", 
               meta="1 bar", 
               reference="Schwieterman et al., 2016",
               path_to_file=os.path.join(path,"photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_GJ876_highO2_10bar" : "photo_clima_p010bar_3.6e-4CO2_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="GJ 876", 
               planet="FP Earth", 
               description="High O2", 
               meta="10 bar", 
               reference="Schwieterman et al., 2016",
               path_to_file=os.path.join(path,"photo_clima_p010bar_3.6e-4CO2_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_GJ876_highO2_100bar" : "photo_clima_p0100bar_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="GJ 876", 
               planet="FP Earth", 
               description="High O2", 
               meta="100 bar", 
               reference="Schwieterman et al., 2016",
               path_to_file=os.path.join(path,"photo_clima_p0100bar_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_GJ876_O2_harman" : "GJ876_worst_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="GJ 876", 
               planet="FP Earth", 
               description="Photolysis of CO2", 
               meta="", 
               reference="Harman et al., 2015; Schwieterman et al., 2016",
               path_to_file=os.path.join(path,"GJ876_worst_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_ADLeo_Segura2005" : "ADLeo_Segura2005_HITRAN2012_toa.rad"
    spectrum(observation="Direct", 
               star="AD Leo", 
               planet="Earth", 
               description="Segura", 
               meta="", 
               reference="Schwieterman 2016; Segura 2005",
               path_to_file=os.path.join(path,"ADLeo_Segura2005_HITRAN2012_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_high O2_10bar_dry" : "profile_o2lb_10bar_dry.pt_filtered_hitran2012_50_100000cm_toa.rad"
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="High O2 Dry", 
               meta="10 bar", 
               reference="Meadows et al., 2017; Schwieterman 2016; Luger & Barnes 2015",
               path_to_file=os.path.join(path,"profile_o2lb_10bar_dry.pt_filtered_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_high O2_10bar_wet" : "profile_o2lb_10bar_h2o.pt_filtered_hitran2012_50_100000cm_toa.rad"
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="High O2 Wet", 
               meta="10 bar", 
               reference="Meadows et al., 2017; Schwieterman 2016; Luger & Barnes 2015",
               path_to_file=os.path.join(path,"profile_o2lb_10bar_h2o.pt_filtered_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_O2 CO2_10bar" : "profile_O2_CO2_10bar_prox_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Evolved O2, CO2", 
               meta="10 bar", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path,"profile_O2_CO2_10bar_prox_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_O2 CO2_90bar" : "profile_O2_CO2_90bar_prox_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Evolved O2, CO2", 
               meta="90 bar", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path,"profile_O2_CO2_90bar_prox_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Venus_10bar_clouds" : "fig17_smart_spectra_pandora10bar_cloudy_500_100000cm-1_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Venus-like", 
               meta="10 bar; Cloudy", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path,"fig17_smart_spectra_pandora10bar_cloudy_500_100000cm-1_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Venus_90bar_clouds" : "fig17_smart_spectra_pandora90bar_clouds_500_100000cm-1_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Venus-like", 
               meta="90 bar; Cloudy", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "fig17_smart_spectra_pandora90bar_clouds_500_100000cm-1_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Gao_1bar" : "smart_gao_1bar_update_xsec_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="FP Earth", 
               meta="1 bar; CO2/O2/CO", 
               reference="Meadows et al., 2017; Gao et al., 2015",
               path_to_file=os.path.join(path, "smart_gao_1bar_update_xsec_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Archean_No haze" : "fig21_HAZE_msun21_0.0Ga_1.00e-02ch4_rmix_5.0E-2__30.66fscale_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Archean", 
               meta="No Haze", 
               reference="Meadows et al., 2017; Arney et al., 2016",
               path_to_file=os.path.join(path, "fig21_HAZE_msun21_0.0Ga_1.00e-02ch4_rmix_5.0E-2__30.66fscale_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Archean_Haze" : "fig21_HAZE_msun21_0.0Ga_3.00e-02ch4_rmix_5.0E-2__30.66fscale_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Archean", 
               meta="Haze", 
               reference="Meadows et al., 2017; Arney et al., 2016",
               path_to_file=os.path.join(path, "fig21_HAZE_msun21_0.0Ga_3.00e-02ch4_rmix_5.0E-2__30.66fscale_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Earth_Cirrus" : "profile_earth_prox.pt_cirrus_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Earth", 
               meta="Cirrus Cloud", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "profile_earth_prox.pt_cirrus_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Earth_Clear" : "profile_earth_prox.pt_filtered_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Earth", 
               meta="Clear", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "profile_earth_prox.pt_filtered_hitran2012_50_100000cm_toa.rad")
              ),
    
    #"Direct_Proxima Centauri_Earth_Stratocumulus" : "profile_earth_prox.pt_stratocum_hitran2012_50_100000cm_toa.rad",
    spectrum(observation="Direct", 
               star="Proxima Centauri", 
               planet="b", 
               description="Earth", 
               meta="Stratocumulus Cloud", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "profile_earth_prox.pt_stratocum_hitran2012_50_100000cm_toa.rad")
              ),
    
    #############################
    # TRANSMISSION SPECTROSCOPY #
    #############################
    
    #"Transmission_GJ876_highO2_1bar" : "photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered.tran",
    spectrum(observation="Transmission", 
               star="GJ 876", 
               planet="FP Earth", 
               description="High O2", 
               meta="1 bar", 
               reference="Schwieterman et al., 2016",
               path_to_file=os.path.join(path, "photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered.tran")
              ),
    
    #"Transmission_GJ876_highO2_10bar" : "photo_clima_p010bar_3.6e-4CO2_GJ876_dry.pt_fixed_filtered.tran",
    spectrum(observation="Transmission", 
               star="GJ 876", 
               planet="FP Earth", 
               description="High O2", 
               meta="10 bar", 
               reference="Schwieterman et al., 2016",
               path_to_file=os.path.join(path, "photo_clima_p010bar_3.6e-4CO2_GJ876_dry.pt_fixed_filtered.tran")
              ),
    
    #"Transmission_GJ876_highO2_10bar" : "photo_clima_p0100bar_GJ876_dry.pt_fixed_filtered.tran",
    spectrum(observation="Transmission", 
               star="GJ 876", 
               planet="FP Earth", 
               description="High O2", 
               meta="100 bar", 
               reference="Schwieterman et al., 2016",
               path_to_file=os.path.join(path, "photo_clima_p0100bar_GJ876_dry.pt_fixed_filtered.tran")
              ),
    
    #"Transmission_GJ876_O2_harman" : "GJ_876_highCO_hitran2012_50_100000cm.tran",
    spectrum(observation="Transmission", 
               star="GJ 876", 
               planet="FP Earth", 
               description="Photolysis of CO2", 
               meta="", 
               reference="Harman et al., 2015; Schwieterman et al., 2016",
               path_to_file=os.path.join(path, "GJ_876_highCO_hitran2012_50_100000cm.tran")
              ),
    
    #"Transmission_ADLeo_Segura2005" : "smartin_adleo_hyak_hitran2012_50_100000cm.tran",
    spectrum(observation="Transmission", 
               star="AD Leo", 
               planet="Earth", 
               description="Segura", 
               meta="", 
               reference="Schwieterman 2016; Segura 2005",
               path_to_file=os.path.join(path, "smartin_adleo_hyak_hitran2012_50_100000cm.tran")
              ),
    
    #"Transmission_Proxima Centauri_O2_10bar_dry" : "profile_o2lb_10bar_dry.pt_filtered_transit.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="High O2 Dry", 
               meta="10 bar", 
               reference="Meadows et al., 2017; Schwieterman 2016; Luger & Barnes 2015",
               path_to_file=os.path.join(path, "profile_o2lb_10bar_dry.pt_filtered_transit.tran")
              ),
    
    #"Transmission_Proxima Centauri_O2_10bar_wet" : "profile_o2lb_10bar_h2o.pt_filtered_transit.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="High O2 Wet", 
               meta="10 bar", 
               reference="Meadows et al., 2017; Schwieterman 2016; Luger & Barnes 2015",
               path_to_file=os.path.join(path, "profile_o2lb_10bar_h2o.pt_filtered_transit.tran")
              ),
    
    #"Transmission_Proxima Centauri_O2 CO2_10bar" : "profile_O2_CO2_10bar_prox_transit_hitran2012_50_100000cm.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Evolved O2, CO2", 
               meta="10 bar", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "profile_O2_CO2_10bar_prox_transit_hitran2012_50_100000cm.tran")
              ),
    
    #"Transmission_Proxima Centauri_O2 CO2_90bar" : "profile_O2_CO2_90bar_prox_transit_hitran2012_50_100000cm.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Evolved O2, CO2", 
               meta="90 bar", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "profile_O2_CO2_90bar_prox_transit_hitran2012_50_100000cm.tran")
              ),
    
    #"Transmission_Proxima Centauri_Venus_10bar_clouds" : "fig24_tran_smart_spectra_pandora10bar_cloudy_500_100000cm-1.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Venus-like", 
               meta="10 bar; Cloudy", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "fig24_tran_smart_spectra_pandora10bar_cloudy_500_100000cm-1.tran")
              ),
    
    #"Transmission_Proxima Centauri_Venus_90bar_clouds" : "fig24_tran_smart_spectra_pandora90bar_clouds_500_100000cm-1.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Venus-like", 
               meta="90 bar; Cloudy", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "fig24_tran_smart_spectra_pandora90bar_clouds_500_100000cm-1.tran")
              ),
    
    #"Transmission_Proxima Centauri_Gao_1bar" : "Gao2015_case3.pt_filtered_transit.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="FP Earth", 
               meta="1 bar; CO2/O2/CO", 
               reference="Meadows et al., 2017; Gao et al., 2015",
               path_to_file=os.path.join(path, "smart_gao_1bar_update_xsec.trnst")
              ),
    
    #"Transmission_Proxima Centauri_Archean_No haze" : "fig28_HAZE_msun21_0.0Ga_1.00e-02ch4_rmix_5.0E-2__30.66fscaleTRAN.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Archean", 
               meta="No Haze", 
               reference="Meadows et al., 2017; Arney et al., 2016",
               path_to_file=os.path.join(path, "fig28_HAZE_msun21_0.0Ga_1.00e-02ch4_rmix_5.0E-2__30.66fscaleTRAN.tran")
              ),
    
    #"Transmission_Proxima Centauri_Archean_Haze" : "fig28_HAZE_msun21_0.0Ga_3.00e-02ch4_rmix_5.0E-2__30.66fscaleTRAN.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Archean", 
               meta="Haze", 
               reference="Meadows et al., 2017; Arney et al., 2016",
               path_to_file=os.path.join(path, "fig28_HAZE_msun21_0.0Ga_3.00e-02ch4_rmix_5.0E-2__30.66fscaleTRAN.tran")
              ),
    
    #"Transmission_Proxima Centauri_Earth_Clear" : "profile_earth_prox.pt_filtered_transit_hitran2012_50_100000cm.tran",
    spectrum(observation="Transmission", 
               star="Proxima Centauri", 
               planet="b", 
               description="Earth", 
               meta="Clear", 
               reference="Meadows et al., 2017",
               path_to_file=os.path.join(path, "profile_earth_prox.pt_filtered_transit_hitran2012_50_100000cm.tran")
              ),
]

In [298]:
for s in spectra: print s.tag

Direct_GJ876_FPEarth_HighO2_1bar
Direct_GJ876_FPEarth_HighO2_10bar
Direct_GJ876_FPEarth_HighO2_100bar
Direct_GJ876_FPEarth_PhotolysisofCO2_
Direct_ADLeo_Earth_Segura_
Direct_ProximaCentauri_b_HighO2Dry_10bar
Direct_ProximaCentauri_b_HighO2Wet_10bar
Direct_ProximaCentauri_b_EvolvedO2,CO2_10bar
Direct_ProximaCentauri_b_EvolvedO2,CO2_90bar
Direct_ProximaCentauri_b_Venus-like_10bar;Cloudy
Direct_ProximaCentauri_b_Venus-like_90bar;Cloudy
Direct_ProximaCentauri_b_FPEarth_1bar;CO2/O2/CO
Direct_ProximaCentauri_b_Archean_NoHaze
Direct_ProximaCentauri_b_Archean_Haze
Direct_ProximaCentauri_b_Earth_CirrusCloud
Direct_ProximaCentauri_b_Earth_Clear
Direct_ProximaCentauri_b_Earth_StratocumulusCloud
Transmission_GJ876_FPEarth_HighO2_1bar
Transmission_GJ876_FPEarth_HighO2_10bar
Transmission_GJ876_FPEarth_HighO2_100bar
Transmission_GJ876_FPEarth_PhotolysisofCO2_
Transmission_ADLeo_Earth_Segura_
Transmission_ProximaCentauri_b_HighO2Dry_10bar
Transmission_ProximaCentauri_b_HighO2Wet_10bar
Transmission_Pro

In [322]:
def write_spectra_csv(spectra, savename="test.csv", lammin=0.1, lammax=20.0):
    """
    """
    wl = []
    wn = []
    toaf = []
    starf = []
    galb = []
    ftrn = []
    absrad = []
    tag = []
    
    for s in spectra:
        mask = (s.data.wavelength >= lammin) & (s.data.wavelength <= lammax)
        tmp = np.chararray(np.sum(mask), itemsize=len(s.tag))
        tmp[:] = s.tag
        tag = np.hstack([tag,tmp])
        wl = np.hstack([wl, s.data.wavelength[mask]])
        wn = np.hstack([wn, s.data.wavenumber[mask]])
        toaf = np.hstack([toaf, s.data.toa_flux[mask]])
        starf = np.hstack([starf, s.data.star_flux[mask]])
        galb = np.hstack([galb, s.data.geo_albedo[mask]])
        ftrn = np.hstack([ftrn, s.data.flux_transmission[mask]])
        absrad = np.hstack([absrad, s.data.absorbing_radius[mask]])
    
    data = np.array([tag, wl, wn, toaf, starf, galb, ftrn, absrad]).T
    
    cols = [
        "File Name",
        "Wavelength",
        "Wavenumber",
        "Top of Atmosphere Flux",
        "Stellar Flux",
        "Geometric Albedo",
        "Flux Transmission",
        "Absorbing Radius"
    ]
    
    df = pd.DataFrame(data, columns=cols)
    
    df.to_csv(savename)

    return

In [323]:
write_spectra_csv(spectra)

In [274]:
# Dictionary containing filenames
rad_file_names = {
    #"Direct_GJ876_highO2_1bar" : "photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
    #"Direct_GJ876_highO2_10bar" : "photo_clima_p010bar_3.6e-4CO2_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
    #"Direct_GJ876_highO2_100bar" : "photo_clima_p0100bar_GJ876_dry.pt_fixed_filtered_hitran2012_50_100000cm_toa.rad",
    #"Direct_GJ876_O2_harman" : "GJ876_worst_hitran2012_50_100000cm_toa.rad",
    #"Direct_ADLeo_Segura2005" : "ADLeo_Segura2005_HITRAN2012_toa.rad",
    #"Direct_Proxima Centauri_high O2_10bar_dry" : "profile_o2lb_10bar_dry.pt_filtered_hitran2012_50_100000cm_toa.rad",
    #"Direct_Proxima Centauri_high O2_10bar_wet" : "profile_o2lb_10bar_h2o.pt_filtered_hitran2012_50_100000cm_toa.rad",
    #"Direct_Proxima Centauri_O2 CO2_10bar" : "profile_O2_CO2_10bar_prox_hitran2012_50_100000cm_toa.rad",
    #"Direct_Proxima Centauri_O2 CO2_90bar" : "profile_O2_CO2_90bar_prox_hitran2012_50_100000cm_toa.rad",
    #"Direct_Proxima Centauri_Venus_10bar_clouds" : "fig17_smart_spectra_pandora10bar_cloudy_500_100000cm-1_toa.rad",
    #"Direct_Proxima Centauri_Venus_90bar_clouds" : "fig17_smart_spectra_pandora90bar_clouds_500_100000cm-1_toa.rad",
    #"Direct_Proxima Centauri_Gao_1bar" : "gao_1bar_final_toa_old.rad",
    #"Direct_Proxima Centauri_Archean_No haze" : "fig21_HAZE_msun21_0.0Ga_1.00e-02ch4_rmix_5.0E-2__30.66fscale_toa.rad",
    #"Direct_Proxima Centauri_Archean_Haze" : "fig21_HAZE_msun21_0.0Ga_3.00e-02ch4_rmix_5.0E-2__30.66fscale_toa.rad",
    #"Direct_Proxima Centauri_Earth_Cirrus" : "profile_earth_prox.pt_cirrus_hitran2012_50_100000cm_toa.rad",
    #"Direct_Proxima Centauri_Earth_Clear" : "profile_earth_prox.pt_filtered_hitran2012_50_100000cm_toa.rad",
    #"Direct_Proxima Centauri_Earth_Stratocumulus" : "profile_earth_prox.pt_stratocum_hitran2012_50_100000cm_toa.rad",
}

In [275]:
trans_file_names = {
    #"Transmission_GJ876_highO2_1bar" : "photo_clima_p01bar_3.6e-4_GJ876_dry.pt_fixed_filtered.tran",
    #"Transmission_GJ876_highO2_10bar" : "photo_clima_p010bar_3.6e-4CO2_GJ876_dry.pt_fixed_filtered.tran",
    #"Transmission_GJ876_highO2_10bar" : "photo_clima_p0100bar_GJ876_dry.pt_fixed_filtered.tran",
    #"Transmission_GJ876_O2_harman" : "GJ_876_highCO_hitran2012_50_100000cm.tran",
    #"Transmission_ADLeo_Segura2005" : "smartin_adleo_hyak_hitran2012_50_100000cm.tran",
    #"Transmission_Proxima Centauri_O2_10bar_dry" : "profile_o2lb_10bar_dry.pt_filtered_transit.tran",
    #"Transmission_Proxima Centauri_O2_10bar_wet" : "profile_o2lb_10bar_h2o.pt_filtered_transit.tran",
    #"Transmission_Proxima Centauri_O2 CO2_10bar" : "profile_O2_CO2_10bar_prox_transit_hitran2012_50_100000cm.tran",
    #"Transmission_Proxima Centauri_O2 CO2_90bar" : "profile_O2_CO2_90bar_prox_transit_hitran2012_50_100000cm.tran",
    #"Transmission_Proxima Centauri_Venus_10bar_clouds" : "fig24_tran_smart_spectra_pandora10bar_cloudy_500_100000cm-1.tran",
    #"Transmission_Proxima Centauri_Venus_90bar_clouds" : "fig24_tran_smart_spectra_pandora90bar_clouds_500_100000cm-1.tran",
    #"Transmission_Proxima Centauri_Gao_1bar" : "Gao2015_case3.pt_filtered_transit.tran",
    #"Transmission_Proxima Centauri_Archean_No haze" : "fig28_HAZE_msun21_0.0Ga_1.00e-02ch4_rmix_5.0E-2__30.66fscaleTRAN.tran",
    #"Transmission_Proxima Centauri_Archean_Haze" : "fig28_HAZE_msun21_0.0Ga_3.00e-02ch4_rmix_5.0E-2__30.66fscaleTRAN.tran",
    #"Transmission_Proxima Centauri_Earth_Clear" : "profile_earth_prox.pt_filtered_transit_hitran2012_50_100000cm.tran",
}

In [135]:
rad_data = {}
for key, value in rad_file_names.iteritems(): 
    fn = os.path.join(path,value)
    wl, wno, solar_spec, TOA_flux, rad_streams = rs.rad(fn)
    tmp_dict = {
        "wl" : wl,
        "wno" : wno,
        "solar_spec" : solar_spec,
        "TOA_flux" : TOA_flux,
        "reflect" : TOA_flux / solar_spec
    }
    rad_data[key] = tmp_dict

In [136]:
trans_data = {}
for key, value in trans_file_names.iteritems(): 
    fn = os.path.join(path,value)
    wl, flux_ratio, absorbing_radius = rs.tran(fn)
    tmp_dict = {
        "wl" : wl,
        "flux_ratio" : flux_ratio,
        "abs_rad" : absorbing_radius
    }
    trans_data[key] = tmp_dict

In [81]:
rad_data["PCb_earth_clear"]["wl"]
trans_data["PCb_earth"]["wl"]

array([  1.99997000e+02,   1.99996000e+02,   1.99995000e+02, ...,
         1.04999999e-01,   1.03999999e-01,   1.02999999e-01])

In [82]:
df = pd.DataFrame.from_dict(rad_data)

In [116]:
lammin=0.3
lammax=1.0

wl=[] 
wno=[] 
TOA_flux=[]
reflect=[]
fnames=[]
for key, value in rad_file_names.iteritems(): 
    fn = os.path.join(path,value)
    twl, twno, tsolar_spec, tTOA_flux, trad_streams = rs.rad(fn)
    mask = (twl > lammin) & (twl < lammax)
    keyarr = np.chararray(len(twl[mask]), itemsize=len(key))
    keyarr[:] = key
    fnames = np.hstack([fnames,keyarr])
    wl = np.hstack([wl,twl[mask]])
    wno = np.hstack([wno,twno[mask]])
    TOA_flux = np.hstack([TOA_flux,tTOA_flux[mask]])
    reflect = np.hstack([reflect,tTOA_flux[mask]/tsolar_spec[mask]])

In [117]:
rad_data = np.array([fnames, wl, wno, TOA_flux, reflect])

In [118]:
rad_data.shape

(5, 396661)

In [119]:
np.savetxt('test_rad.csv',
           rad_data.T,
           delimiter=',',
           header='File Name, Wavelength, Wavenumber, TOA Flux, Geometric Albedo',
           comments='',
           fmt=('%s', '%s', '%s', '%s', '%s')
           )

In [112]:
lammin=0.3
lammax=1.0

wl=[] 
flux_ratio=[] 
absorbing_radius=[]
fnames=[]
for key, value in trans_file_names.iteritems(): 
    fn = os.path.join(path,value)
    twl, tflux_ratio, tabsorbing_radius = rs.tran(fn)
    mask = (twl > lammin) & (twl < lammax)
    keyarr = np.chararray(len(twl[mask]), itemsize=len(key))
    keyarr[:] = key
    fnames = np.hstack([fnames,keyarr])
    wl = np.hstack([wl,twl[mask]])
    flux_ratio = np.hstack([flux_ratio,tflux_ratio[mask]])
    absorbing_radius = np.hstack([absorbing_radius,tabsorbing_radius[mask]])

In [113]:
tran_data = np.array([fnames, wl, flux_ratio, absorbing_radius])

In [114]:
tran_data.shape

(4, 22310)

In [115]:
np.savetxt('test_tran.csv',
           tran_data.T,
           delimiter=',',
           header='File Name, Wavelength, Flux Transmission, Atmosphere Absorbing Radius',
           comments='',
           fmt=('%s', '%s', '%s', '%s')
           )

In [128]:
df = pd.DataFrame(tran_data.T)

In [137]:
lammin=0.3
lammax=1.0

wl=[] 
wno=[] 
TOA_flux=[]
reflect=[]
flux_ratio=[] 
absorbing_radius=[]
fnames=[]

# Loop over rad files
for key, value in rad_file_names.iteritems(): 
    fn = os.path.join(path,value)
    twl, twno, tsolar_spec, tTOA_flux, trad_streams = rs.rad(fn)
    mask = (twl > lammin) & (twl < lammax)
    keyarr = np.chararray(len(twl[mask]), itemsize=len(key))
    keyarr[:] = key
    fnames = np.hstack([fnames,keyarr])
    tmp = np.chararray(len(twl[mask]), itemsize=1)
    tmp[:] = ""
    flux_ratio=np.hstack([flux_ratio, tmp]) 
    absorbing_radius=np.hstack([absorbing_radius,tmp])
    wl = np.hstack([wl,twl[mask]])
    wno = np.hstack([wno,twno[mask]])
    TOA_flux = np.hstack([TOA_flux,tTOA_flux[mask]])
    reflect = np.hstack([reflect,tTOA_flux[mask]/tsolar_spec[mask]])   

In [138]:
# Loop over transit files
for key, value in trans_file_names.iteritems(): 
    fn = os.path.join(path,value)
    twl, tflux_ratio, tabsorbing_radius = rs.tran(fn)
    mask = (twl > lammin) & (twl < lammax)
    keyarr = np.chararray(len(twl[mask]), itemsize=len(key))
    keyarr[:] = key
    fnames = np.hstack([fnames,keyarr])
    tmp = np.ones(len(twl[mask]))
    tmp[:] = ""
    wno = np.hstack([wno,tmp])
    TOA_flux = np.hstack([TOA_flux,tmp])
    reflect = np.hstack([reflect,tmp])  
    wl = np.hstack([wl,twl[mask]])
    flux_ratio = np.hstack([flux_ratio,tflux_ratio[mask]])
    absorbing_radius = np.hstack([absorbing_radius,tabsorbing_radius[mask]])

In [139]:
data = np.array([fnames, wl, wno, TOA_flux, reflect, flux_ratio, absorbing_radius]).T

In [143]:
np.savetxt('test_data.csv',
           data,
           delimiter=',',
           header='File Name, Wavelength, Wavenumber, Top of Atmosphere Flux, Geometric Albedo, Flux Transmission, Atmosphere Absorbing Radius',
           comments='',
           fmt=('%s', '%s', '%s', '%s', '%s', '%s', '%s')
           )

In [142]:
data.shape

(418971, 7)